**Introduction**

The restaurant business is very competitive. Over 50% of restaurants go out of business within the first three years of opening. However, choosing a location that is not already overly saturated with restaurants could help a restaurant survive.

In this project we will source out the best location for a restaurant in Oklahoma City. The best location will have a dense population and low amount of restaurants. We will divide Oklahoma City into nine main sections based on the intersections of the six main highways and choose the best location from the nine sections. 

A company or individual who is looking to open a restaurant in Oklahoma City is the target audience for this project. The results of this project will help make a logical decision on the best place for a new restaurant in Oklahoma City. 

**Data**

We will be using Foursquare API to obtain our data. Since there is no clear definition of neighborhoods or districts for Oklahoma City, the project decides to divide Oklahoma City into nine parts by the intersections of six main highways that pass through Oklahoma City. Then, we will explore the venues of each area by using Foursquare API, which will return venues with coordinates and category. Our goal is to find the largest ratio of number of grocery stores to the number of restaurants in one section which will then be the best location for a new restaurant.


**Methodology**

There are no clear definition of neighborhoods or districts for Oklahoma City, the project decides to divide Oklahoma City into nine parts by the intersections of six main highways that pass through Oklahoma City. Those six main highways are OK74, US77, I35 which pass through OKC from north to south and I40, I44, John Kilpatrick Turnpike which pass through OKC from west to east.The nine intersections of six highways will be counted as the center of corresponding area. The nine sections will be called Middle, South, North, West, East, Southwest, Southwest, Northwest, and Northeast. 

_This project will count the center of midwest city as the new southeast center_

In [15]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner


In [16]:
area = {'Area': ['Middle', 'South', 'North', 'West', 'East', 'Southwest', 'Southeast', 'Northwest', 'Northeast'], 
        'Latitude': [35.529652, 35.462865, 35.599336, 35.511339, 35.538950, 35.461761, 35.464067, 35.610904,35.600369],
        'Longitude': [-97.513947, -97.495157, -97.504152, -97.576386, -97.460976, -97.575795,  -97.406217, -97.585991, -97.432060]}

okc_data = pd.DataFrame(data=area)
okc_data

,Area,Latitude,Longitude
0,Middle,35.529652,-97.513947
1,South,35.462865,-97.495157
2,North,35.599336,-97.504152
3,West,35.511339,-97.576386
4,East,35.538950,-97.460976
5,Southwest,35.461761,-97.575795
6,Southeast,35.464067,-97.406217
7,Northwest,35.610904,-97.585991
8,Northeast,35.600369,-97.432060


**Next we will plot each center on a map using the Folium library**

In [41]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests # library to handle requests
import json # library to handle JSON files

Solving environment: done

# All requested packages already installed.



In [42]:
# use geopy libirary to get coordinates of OKC
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

address = 'Oklahoma City, OK'

geolocator = Nominatim(user_agent="okc_explorer")
location = geolocator.geocode(address)
okc_latitude = location.latitude
okc_longitude = location.longitude
print('The geograpical coordinate of Oklahoma City are {}, {}.'.format(okc_latitude, okc_longitude))

The geograpical coordinate of Oklahoma City are 35.4729886, -97.5170536.


In [55]:
# create map of Oklahoma City using latitude and longitude values
okc_map = folium.Map(location=[okc_latitude, okc_longitude], zoom_start=11)

# add markers to map
for area, lat, lng in zip(okc_data['Area'], okc_data['Latitude'], okc_data['Longitude']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(okc_map)  
    
okc_map

**Now we will use Foursquare API to get venues for each of the nine sections**

In [44]:
CLIENT_ID = 'IIO0CADBGVWOK5XDCSCJHSMZUOIECGGBMTVFH2TEFNJRL2AE' 
CLIENT_SECRET = 'QLRQYIQBGFLQ52WV2M13FTUKJS5HPAFUEW4U4VOML00VVHF5' 
VERSION = '20180605' 

In [46]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
#radius = 500 

def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
# create the API request URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)            

In [47]:
okc_venues=getNearbyVenues(names=okc_data['Area'],
                                   latitudes=okc_data['Latitude'],
                                   longitudes=okc_data['Longitude']
                                  )
                                  

Middle
South
North
West
East
Southwest
Southeast
Northwest
Northeast


In [48]:
print(okc_venues.shape)
okc_venues.head()

(474, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Middle,35.529652,-97.513947,The Shack,35.536220,-97.518536,Seafood Restaurant
1,Middle,35.529652,-97.513947,Bedlam BBQ,35.522051,-97.502132,BBQ Joint
2,Middle,35.529652,-97.513947,Whole Foods Market,35.534002,-97.530522,Grocery Store
3,Middle,35.529652,-97.513947,Vintage Coffee,35.521489,-97.530103,Coffee Shop
4,Middle,35.529652,-97.513947,Republic Gastropub,35.532408,-97.533954,Gastropub


In [49]:
okc_venues.groupby('Area').count()

,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Area,,,,,,
East,27,27,27,27,27,27
Middle,69,69,69,69,69,69
North,15,15,15,15,15,15
Northeast,29,29,29,29,29,29
Northwest,75,75,75,75,75,75
South,96,96,96,96,96,96
Southeast,66,66,66,66,66,66
Southwest,30,30,30,30,30,30
West,67,67,67,67,67,67


In [50]:
print('There are {} uniques categories.'.format(len(okc_venues['Venue Category'].unique())))

There are 156 uniques categories.


**Utilize onehot to get the ratio of restaurants in each of the nine sections**

In [51]:
# one hot encoding
okc_onehot = pd.get_dummies(okc_venues[['Venue Category']], prefix="", prefix_sep="")

# add area column back to dataframe
okc_onehot['Area'] = okc_venues['Area'] 

# move area column to the first column
fixed_columns = [okc_onehot.columns[-1]] + list(okc_onehot.columns[:-1])
okc_onehot = okc_onehot[fixed_columns]

okc_onehot.head()

,Area,ATM,Accessories Store,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Winery,Wings Joint,Women's Store,Zoo Exhibit
0,Middle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Middle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Middle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Middle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Middle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
okc_onehot.shape

(474, 157)

In [53]:
okc_grouped = okc_onehot.groupby('Area').mean().reset_index()

**Results**

In [54]:
pd.DataFrame(data=[okc_data['Area'], okc_grouped['Grocery Store'], okc_grouped['Restaurant']])

,0,1,2,3,4,5,6,7,8
Area,Middle,South,North,West,East,Southwest,Southeast,Northwest,Northeast
Grocery Store,0,0.0289855,0,0,0,0.0104167,0.0454545,0,0.0149254
Restaurant,0,0,0,0,0,0.0416667,0.0151515,0,0.0149254


In the introduction it was stated that the goal of this project was to find the largest ratio between restaurants and grocery stores in order to find the best location for a new restaurant in Oklahoma City. The table above shows the largest ratio is the South with 0.0289855 and 0. 

**Discussion and Conclusion**

This report was able to give a good general location for a new restaurant. In order to get a more accurate and precise location, more analysis on the South section should be made. Other factors could be taken into consideration as well such as, price of rent, average age of demographics in the area, etc. Opening a new restaurant is a large task that takes a lot of prior research and analysis. I would consider the work done in the report to be apart of the initial research done to find a great new location for a restaurant. 